# Factorial Kriging Analysis

<!-- SUMMARY: Filtering images using Factorial Kriging Analysis. Example on a multivariate acquisition  -->

<!-- CATEGORY: Courses -->

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import numpy as np
import os
import urllib.request

flagInternetAvailable = True ## Set to false if no internet

The Grid containing the information is downloaded from the distribution.

The loaded file (called **grid **) contains 3 variables:

- **P** (phosphorus) which is the *variable of interest*
- **Cr** (chromium) is an auxiliary variable
- **Ni** (nickel) another auxiliary variable


In [ ]:
fileImage='Image.ascii'
if flagInternetAvailable:
    fileNF, head = urllib.request.urlretrieve('https://soft.minesparis.psl.eu/gstlearn/data/FKA/'+fileImage,'./'+fileImage)
else:
    fileNF='./'+fileImage

grid = gl.DbGrid.createFromNF(fileNF)
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN, ndim)

In [ ]:
dbfmt = gl.DbStringFormat()
dbfmt.setFlags(flag_resume=False,flag_vars=False,flag_stats=True, names="P")
grid.display(dbfmt)

Note that some pixels are not informed for variable **P**.

Statistics on auxiliary variables

In [ ]:
dbfmt.setFlags(flag_vars=False, flag_resume=True, flag_stats=True, names=["Cr", "Ni"])
grid.display(dbfmt)

Correlation between variables

In [ ]:
ax = gp.correlation(grid, namex="Cr", namey="P", bins=100)

In [ ]:
ax = gp.correlation(grid, namex="Ni", namey="P", bins=100)

In [ ]:
ax = gp.correlation(grid, namex="Ni", namey="Cr", bins=100)

Using inverse square distance for completing the variable **P**

In [ ]:
grid.setLocator("P", gl.ELoc.Z)
err = gl.db_grid_fill(grid)

We concentrate on the variable of interest **P** completed (**Fill.P**) in the next operations

In [ ]:
gp.setDefaultGeographic(dims=[8,8])
ax = grid.plot("Fill.P")

Variogram Calculation along Grid main axes

In [ ]:
varioparam = gl.VarioParam.createMultipleFromGrid(grid, npas=100)
varioP = gl.Vario(varioparam, grid)
err = varioP.compute()
modelP = gl.Model()
err = modelP.fit(varioP, types=[gl.ECov.NUGGET, gl.ECov.SPHERICAL, gl.ECov.POWER],
                 optvar=gl.Option_VarioFit(True, False))
modelP.setDriftIRF(0,0)

In [ ]:
ax = gp.varmod(varioP, modelP)

We must define the **Neighborhood**

In [ ]:
neigh = gl.NeighImage([10,10])

The **image** neighborhood is based on $(2*10+1)^2=441$ pixels (centered on the target pixel).

During the estimation, only the contribution of second and third basic structures are kept (Nugget Effect is filtered out): ** Factorial Kriging Analysis**.

In [ ]:
modelP.setCovaFiltered(0, True)
means = gl.dbStatisticsMono(grid,["Fill.P"],[gl.EStatOption.MEAN]).getValues()
modelP.setMeans(means)
modelP

In [ ]:
err  = gl.krimage(grid,modelP,neigh,namconv=gl.NamingConvention("Mono"))

In [ ]:
ax = grid.plot("Mono*.P")
ax.decoration(title="P denoised (monovariate)")

Correlation for **P** variable between Initial image (completed) and its Filtered version (monovariate FKA)

In [ ]:
grid

In [ ]:
ax = gp.correlation(grid, namex="Fill.P", namey="Mono.Fill.P", bins=100)
ax.decoration(xlabel="P Filled",ylabel="P Filtered (Mono)")

## Multivariate approach

In [ ]:
grid.setLocators(["Fill.P", "Cr", "Ni"], gl.ELoc.Z)

varioM = gl.Vario(varioparam, grid)
err = varioM.compute()
modelM = gl.Model()
err = modelM.fit(varioM, types=[gl.ECov.NUGGET, gl.ECov.SPHERICAL, gl.ECov.POWER],
                 optvar=gl.Option_VarioFit(True, False))
modelM.setDriftIRF(0,0)

In [ ]:
ax = gp.varmod(varioM, modelM)

Printing the contents of the fitted Multivariate Mpdel

In [ ]:
modelM.setCovaFiltered(0, True)
means = gl.dbStatisticsMono(grid,["Fill.P", "Cr", "Ni"],[gl.EStatOption.MEAN]).getValues()
modelM.setMeans(means)
modelM

Multivariable Factorial Kriging Analysis

In [ ]:
err  = gl.krimage(grid,modelM,neigh,namconv=gl.NamingConvention("Multi"))

Note that, using the same **neigh** as in monovariate, the dimension of the Kriging System is now $3 * 441 = 1323$

In [ ]:
ax = grid.plot("Multi*.P")
ax.decoration(title="P denoised (multivariate)")

Correlation for **P** variable between Initial image and its Filtered version (multivariate FKA)

In [ ]:
ax = gp.correlation(grid, namex="Fill.P", namey="Multi.Fill.P", bins=100)
ax.decoration(xlabel="P Filled",ylabel="P Filtered (Multi)")

Correlation for **P** filtered variable between he Monovariate and the Multivariate case

In [ ]:
ax = gp.correlation(grid, namex="Mono.Fill.P", namey="Multi.Fill.P", bins=100)
ax.decoration(xlabel="P Filtered (Mono)",ylabel="P Filtered (Multi)")